# Importation des librairies

In [2]:
import json
import pandas as pd
import numpy as np
import re
import string
import emoji
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import FrenchStemmer
# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('stopwords')
# Préparation des outils
stop_words = set(stopwords.words('french'))
stemmer = FrenchStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\etudiant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\etudiant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import warnings
warnings.filterwarnings('ignore')

# Load the data

In [4]:
data = pd.read_json('comments_f1.json')

data.head()


,_id,name,appreciation,comments
0,{'$oid': '647bab3240fabfafb506263d'},Dahl de lentilles corail,4.5/5,[Recette de base que j'adapte en fonction des ...
1,{'$oid': '647bab3240fabfafb506263e'},Poêlée de panais,4.4/5,[Délicieux. C'est la première fois que je cuis...
2,{'$oid': '647bab3240fabfafb506263f'},Aubergines au four,4.3/5,[Facile à préparer et très sain. Il faut mettr...
3,{'$oid': '647bab3240fabfafb5062640'},Curry de pois chiches,4.8/5,"[après 10' j'ajoute des épinards., Rien à ajou..."
4,{'$oid': '647bab3240fabfafb5062641'},Galettes de pomme de terre,3.9/5,[Très bien. Je n'ai mis qu'un oignon que j'ai ...


# Data preprocessing

## 1- Data Cleaning 

### Detecter les éléments à nettoyer 

In [5]:
def has_url(comment):
    return bool(re.search(r'http[s]?://\S+', comment))

def has_mention(comment):
    return bool(re.search(r'@\w+', comment))

def has_uppercase_words(comment):
    return any(word.isupper() for word in comment.split())

def has_short_words(comment, threshold=3):
    return any(len(word) < threshold for word in comment.split())

def has_emojis(comment):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               "]+", flags=re.UNICODE)
    return bool(emoji_pattern.search(comment))

def has_special_characters(comment):
    return bool(re.search(r'[^\w\s]', comment))

def has_duplicates(comments):
    seen = set()
    for comment in comments:
        if comment in seen:
            return True  # Doublon trouvé
        seen.add(comment)
    return False  # Aucun doublon

# Charger les données JSON
comments = data['comments'].explode().tolist()

# Statistiques de présence
stats = {
    "urls": 0,
    "mentions": 0,
    "uppercase_words": 0,
    "short_words": 0,
    "emojis": 0,
    "special_characters": 0,
    "duplicates": "Non"  # Initialise avec "Non"
}

# Vérification de la présence des éléments dans chaque commentaire
for comment in comments:
    if has_url(comment):
        stats['urls'] += 1
    if has_mention(comment):
        stats['mentions'] += 1
    if has_uppercase_words(comment):
        stats['uppercase_words'] += 1
    if has_short_words(comment):
        stats['short_words'] += 1
    if has_emojis(comment):
        stats['emojis'] += 1
    if has_special_characters(comment):
        stats['special_characters'] += 1

# Vérification des doublons
stats['duplicates'] = "Oui" if has_duplicates(comments) else "Non"

# Affichage des résultats
print("Présence dans les commentaires :")
for key, value in stats.items():
    if key == "duplicates":
        print(f"Doublons présents : {value}")
    else:
        print(f"{key.capitalize().replace('_', ' ')}: {value} occurrences")

Présence dans les commentaires :
Urls: 4 occurrences
Mentions: 0 occurrences
Uppercase words: 2063 occurrences
Short words: 21678 occurrences
Emojis: 320 occurrences
Special characters: 22202 occurrences
Doublons présents : Oui


### Supprimer les problèmes detectés 

In [6]:
def remove_urls(comment):
    """Supprime les URLs d'un commentaire."""
    return re.sub(r'http[s]?://\S+', '', comment)

def remove_emojis(comment):
    """Supprime tous les emojis d'un commentaire."""
    return emoji.replace_emoji(comment, replace='')

def clean_punctuation(sentence):
    """ Remplace certaines formes de ponctuation par des espaces et supprime d'autres, tout en préservant les apostrophes dans les contractions. """
    # Suppression de la ponctuation sauf les apostrophes utilisées dans les contractions
    sentence = re.sub(r'(?<!\w)[\'\"](?!\w)', '', sentence)  # Supprime les apostrophes et guillemets qui ne sont pas entourés par des lettres
    sentence = re.sub(r'[?|!|:|;|.|,|)|(|\|/]', ' ', sentence)  # Remplace d'autres ponctuations par des espaces
    sentence = sentence.replace("\n", " ")
    return sentence.strip()

def keep_alpha(sentence):
    """Conserve uniquement les caractères alphabétiques et gère certains caractères spéciaux."""
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-ZÀ-ÿ\']+', '', word)  # Inclure les caractères accentués français et apostrophes
        alpha_word = alpha_word.replace('œ', 'oe')  # Remplacer 'œ' par 'oe'
        alpha_sent += alpha_word + " "
    return alpha_sent.strip()

def remove_repeated_characters(comment):
    """Réduit les caractères répétés dans les mots pour éviter des variations comme 'délicieeuuuuux' à 'délicieux'."""
    return re.sub(r'(\w)(\1{2,})', r'\1', comment)

# Chargement et préparation des données
comments = data['comments'].explode().drop_duplicates().tolist()

# Application du nettoyage
cleaned_comments = []
for comment in comments:
    comment = remove_urls(comment)
    comment = remove_emojis(comment)
    comment = clean_punctuation(comment)
    comment = keep_alpha(comment)
    comment = remove_repeated_characters(comment)
    cleaned_comments.append(comment)
    
# Création d'un DataFrame à partir des commentaires nettoyés
cleaned_data = pd.DataFrame(cleaned_comments, columns=['cleaned_comments'])

# Afficher les premiers éléments nettoyés pour vérifier
print(cleaned_data.head())

                                    cleaned_comments
0  Recette de base que j'adapte en fonction des l...
1  Noubliez pas de faire cuire en avance les caro...
2  D'accord pour le commentaire sur les carottes ...
3  Très bonne recette réalisée avec des lentilles...
4              Facile à faire plat délicieux et sain


## 2- Preprocessing 

In [7]:
def tokenize_sentences(text):
    """Tokenise le texte en mots."""
    return word_tokenize(text, language='french')

def remove_stop_words(tokens):
    """Enlève les stop words de la liste de tokens."""
    return [word for word in tokens if word not in stop_words]

def stem_words(words):
    """Applique le stemming sur la liste de mots."""
    return [stemmer.stem(word) for word in words]

def labelize_scores(score):
    """Convertit un score textuel en une étiquette numérique simple."""
    return int(round(float(score.split('/')[0])))

def preprocess_text(text):
    """Applique le tokenization, la suppression des stop words, et le stemming."""
    tokens = tokenize_sentences(text)
    filtered_tokens = remove_stop_words(tokens)
    stemmed_tokens = stem_words(filtered_tokens)
    return stemmed_tokens


In [12]:
# Chargement des données nettoyées
cleaned_comments = cleaned_data['cleaned_comments'].explode().drop_duplicates().tolist()

# Application du prétraitement
preprocessed_comments = []
for comment in cleaned_comments:
    processed_text = preprocess_text(comment)
    # Joindre les tokens pour former une chaîne de caractères avant de les ajouter à la liste
    preprocessed_comments.append(' '.join(processed_text))

# Création d'un DataFrame à partir des commentaires prétraités
preprocessed_data = pd.DataFrame(preprocessed_comments, columns=['preprocessed_comments'])

# Application de la fonction labelize_scores à la colonne des scores
data['appreciation'] = data['appreciation'].apply(labelize_scores) 

# Appliquer la transformation directement sur la colonne 'appreciation'
data['appreciation'] = data['appreciation'].apply(lambda x: int(round(float(x.split('/')[0]))))


# Affichage des premiers éléments prétraités pour vérifier
print(preprocessed_data.head())

# Affichage des scores transformés pour vérifier
print(data[['appreciation']])

                               preprocessed_comments
0  recet bas j'adapt fonction légum maison c'est ...
1  noubl fair cuir avanc carott sinon recet tres ...
2  d'accord commentair carott je n'ai trouv ça re...
3  tres bon recet réalis lentill vert davantag la...
4                        facil fair plat délici sain
     appreciation
0               4
1               4
2               4
3               5
4               4
..            ...
360             4
361             5
362             4
363             4
364             5

[365 rows x 1 columns]


In [14]:
if '_id' in data.columns:
    data.drop('_id', axis=1, inplace=True)
data.head()

,name,appreciation,comments,label
0,Dahl de lentilles corail,4,[Recette de base que j'adapte en fonction des ...,4
1,Poêlée de panais,4,[Délicieux. C'est la première fois que je cuis...,4
2,Aubergines au four,4,[Facile à préparer et très sain. Il faut mettr...,4
3,Curry de pois chiches,5,"[après 10' j'ajoute des épinards., Rien à ajou...",5
4,Galettes de pomme de terre,4,[Très bien. Je n'ai mis qu'un oignon que j'ai ...,4


In [15]:
data.to_json("Cleaned_data.json", orient='records', lines=True)

In [16]:
with open('Cleaned_data.json', 'r', encoding='utf-8') as f:
    data_loaded = json.load(f)


JSONDecodeError: Extra data: line 2 column 1 (char 6808)